In [1]:
!pip install geemap >> /dev/null
!pip install pycrs >> /dev/null

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ee
import geemap

In [3]:
ee.Authenticate()
ee.Initialize(project='ee-testgee')

In [4]:
studyArea = ee.Geometry.Rectangle(98.82, 7.98, 99.02, 8.18)
KrabiCol = ee.ImageCollection('COPERNICUS/S2_SR').filterDate('2020-01-01','2020-04-30').filterBounds(studyArea)
KrabiImg = KrabiCol.median().clip(studyArea).select(['B4','B3','B2', 'B8'])
Map = geemap.Map(center = [8.08,98.92], zoom = 12)
vis_Para = {'min': 0.0, 'max': 1000, 'bands':['B4','B3','B2']}
Map.addLayer(KrabiImg, vis_Para, name="Krabi RGB Image")
Map

Map(center=[8.08, 98.92], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [6]:
lc_gt_train = geemap.shp_to_ee('/content/krabi_gt_train.shp')
example_pnts = KrabiImg.sampleRegions(lc_gt_train, scale=30) #get band values for points from raster

In [7]:
example_pnts


In [8]:
myClassifier = ee.Classifier.minimumDistance().train(example_pnts, 'id')
KrabiClassImg= KrabiImg.classify(myClassifier)
Map = geemap.Map(center = [8.08,98.92], zoom = 12)
vis_Para = {'min': 1, 'max': 4, 'palette':['green', 'red', 'blue', 'yellow']}
Map.addLayer(KrabiClassImg, vis_Para, name="Krabi - Landcover")
Map

Map(center=[8.08, 98.92], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [9]:
myClassifierRF = ee.Classifier.smileRandomForest(numberOfTrees=100).train(example_pnts, 'id')
KrabiClassImg= KrabiImg.classify(myClassifierRF)
Map = geemap.Map(center = [8.08,98.92], zoom = 12)
vis_Para = {'min': 1, 'max': 4, 'palette':['green', 'red', 'blue', 'yellow']}
Map.addLayer(KrabiClassImg, vis_Para, name="Krabi - Landcover")
Map

Map(center=[8.08, 98.92], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [10]:
lc_gt_val = geemap.shp_to_ee('/content/krabi_gt_val.shp')
validation_pnts = KrabiImg.sampleRegions(lc_gt_val, scale=30)

In [12]:
validation_class = validation_pnts.classify(myClassifier)
print(validation_class.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'B2': 252.5, 'B3': 438, 'B4': 250, 'B8': 3224.5, 'classification': 1, 'id': 1}}


In [13]:
val_accuracy = validation_class.errorMatrix('id', 'classification')
print(val_accuracy.getInfo())
print(val_accuracy.accuracy().getInfo())
print(val_accuracy.kappa().getInfo())

[[0, 0, 0, 0, 0], [0, 25, 0, 2, 0], [0, 4, 24, 0, 2], [0, 1, 0, 20, 0], [0, 2, 1, 0, 36]]
0.8974358974358975
0.8614154575066626


In [14]:
validation_class = validation_pnts.classify(myClassifierRF)
print(validation_class.first().getInfo())
val_accuracy = validation_class.errorMatrix('id', 'classification')
print(val_accuracy.getInfo())
print(val_accuracy.accuracy().getInfo())
print(val_accuracy.kappa().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'B2': 252.5, 'B3': 438, 'B4': 250, 'B8': 3224.5, 'classification': 1, 'id': 1}}
[[0, 0, 0, 0, 0], [0, 25, 0, 2, 0], [0, 0, 28, 0, 2], [0, 1, 0, 20, 0], [0, 0, 2, 0, 37]]
0.9401709401709402
0.9189429928741093
